# Mini-Project I
During this project, we will practice handling of complex lists and dictionaries in Python. Plus, we will learn how to work with API documentation. Don't be afraid to search for the information in the [**documentation**](https://api.tfl.gov.uk/swagger/ui/index.html?url=/swagger/docs/v1#!/AccidentStats/AccidentStats_Get).

Let's go to the tasks, we have some parsing to do :)!!

In [46]:
# import packages we need (remember what packages we used yesterday during the API session)
from dotenv import load_dotenv
from collections import Counter
import requests, os, json

load_dotenv()

True

In [47]:
# URLs
url_air_quality = "https://api.tfl.gov.uk/AirQuality"
url_modes = 'https://api.tfl.gov.uk/Line/Meta/Modes'
url_bike_point = 'https://api.tfl.gov.uk/BikePoint/'
url_lines_tube = 'https://api.tfl.gov.uk/Line/Mode/tube'
url_lines_bus = 'https://api.tfl.gov.uk/Line/Mode/bus'
url_victoria = 'https://api.tfl.gov.uk/Line/victoria/StopPoints'

## Demo

In [48]:
# append this authorization to the request url
def authorization():
    app_id = os.getenv("TFL_API_ID")
    app_key = os.getenv("TFL_API_KEY")
    url_append = f'?app_id={app_id}&app_key={app_key}'
    return url_append

In [49]:
# send the request to the API
# check if the request was successful
def request_data(url, params='', auth=True, type='json'):
    if auth == 'True':
        res = requests.get(url + authorization(), params)
        res.status_code
    else:
        res = requests.get(url, params)
        res.status_code
        
    if type == 'text':
        return res.text

    return res.json()

In [50]:
# unpack dictionary in list,
# then parse for key and output its values as list
def unpack_to_list(tfl_data, field):

    data_list = []
    
    for type in tfl_data:
        for key, value in type.items():
            if key == field:
                data_list += [value]
    
    return data_list

In [51]:
# remove any unwanted fields in a dictionary output,
# then output the result
def skip_fields(dictionary, fields_to_skip):
    for key, value in dictionary.items():
        flag = False

        for field in fields_to_skip:
            if key == field:
                flag = True
        
        if flag != True:
            print(f'{key} : {value}')

In [52]:
# request data and export the information that was returned using .json()
forecast_data = request_data(url_air_quality)

## Task
Parse the dictionary and print the AirQuality predictions for tomorrow

In [53]:
# take relevant data from future
forecast_future = forecast_data['currentForecast'][1]

# data to remove from output
ignore_fields = ['$id', '$type', 'forecastID', ]

# remove irrelevant data and output the rest
skip_fields(forecast_future, ignore_fields)

forecastType : Future
publishedDate : 2022-08-04T11:02:33Z
fromDate : 2022-09-04T00:00:00Z
toDate : 2022-09-04T23:59:00Z
forecastBand : Low
forecastSummary : Low air pollution forecast valid from Saturday 9 April to end of Saturday 9 April GMT
nO2Band : Low
o3Band : Low
pM10Band : Low
pM25Band : Low
sO2Band : Low
forecastText : Following a cold start to the weekend, temperatures are forecast to warm up by Monday.  Saturday is expected to be mainly dry with some sun in parts.&lt;br/&gt;&lt;br/&gt;A continuing northerly air feed to Greater London should be relatively &#39;clean&#39; and carry little by way of imported pollution. There should also be enough air movement to prevent the build-up of local emissions. &lt;br/&gt;&lt;br/&gt;Overall air pollution is expected to remain &#39;Low&#39; throughout the forecast period for the following pollutants:&lt;br/&gt;&lt;br/&gt;Nitrogen Dioxide&lt;br/&gt;Ozone&lt;br/&gt;PM10 Particulate&lt;br/&gt;PM2.5 Particulate&lt;br/&gt;Sulphur Dioxide&lt;b

## Task
What are the different modes of transport which are operated by Transfer for London? How many of modes do they have?

Print the list with different modes of transport, plus their count. Example output:
```
[bus, cable-car,.....]
Number of different modes of transport is: xyz
```

We need to search the documentation for correct request.

In [54]:
# request data for modes
modes_data = request_data(url_modes)

In [55]:
# convert list into nested dictionary and output 'modeName' field as a list
print(unpack_to_list(modes_data, 'modeName'))

# print number of modes
print(f'Number of different modes of transport: {len(modes_data)}')

['bus', 'cable-car', 'coach', 'cycle', 'cycle-hire', 'dlr', 'elizabeth-line', 'interchange-keep-sitting', 'interchange-secure', 'national-rail', 'overground', 'replacement-bus', 'river-bus', 'river-tour', 'taxi', 'tflrail', 'tram', 'tube', 'walking']
Number of different modes of transport: 19


## Task
How many BikePoints in London are operated by Transfor for London? How many docks are in **all** BikePoints? There is the information for empty and full docks for each BikePoint.

In [56]:
# request data for bike points and return as .json
bike_points_data = request_data(url_bike_point)

In [57]:
# count the number of bike points
bike_points = unpack_to_list(bike_points_data, 'id')
print(f'There are {len(bike_points)} BikePoints in London operated by TFL.')

There are 791 BikePoints in London operated by TFL.


In [58]:
# isolate additional properties from the fields under each BikePoint,
# and aggregate them in a list
add_props = []
for num in range(len(bike_points_data)):
    for key,value in bike_points_data[num].items():
        if key == 'additionalProperties':
            add_props += value

In [59]:
# search for keywords 'NbDocks' and NbEmptyDocks' and list their indices
index = []
count = 0
for description in add_props:
    for key, value in description.items():
        if key == 'key' and value == 'NbDocks':
            index.append(count)
        elif key == 'key' and value == 'NbEmptyDocks':
            index.append(count)
    count += 1

# using the indices, aggregate the values (counts)
# of all empty and non-empty docks into a list
docks = []
for i in index:
    for key, value in add_props[i].items():
        if key == 'value':
            docks += [value]

# convert the collected string values into integers
# and perform a sum to get a total value
dock_count = sum(map(int, docks))

print(f'There are {dock_count} docks in all BikePoints.')

There are 30665 docks in all BikePoints.


## Task
How many tube and bus lines are in London? Print names of all tube lines.

In [60]:
# request data for all tube and bus lines
lines_tube_data = request_data(url_lines_tube)
lines_bus_data = request_data(url_lines_bus)

In [61]:
# get list of all tube and bus lines
tube_names = unpack_to_list(lines_tube_data, 'name')
bus_names = unpack_to_list(lines_bus_data, 'name')

In [62]:
# check for intesection between bus names & tube lines
same_names = set(tube_names) & set(bus_names)

if list(same_names) == []:
    print('There are no lines shared between tubes and bus modes.')
else:
    for same in same_names:
        print(same)

There are no lines shared between tubes and bus modes.


In [63]:
# add both lines to calculate total regardless of mode
total_lines = len(bus_names) + len(tube_names)
print(f'There are a total of {total_lines} tube and bus lines in London.')

There are a total of 688 tube and bus lines in London.


In [64]:
# print names from all tube lines
tube_names

['Bakerloo',
 'Central',
 'Circle',
 'District',
 'Hammersmith & City',
 'Jubilee',
 'Metropolitan',
 'Northern',
 'Piccadilly',
 'Victoria',
 'Waterloo & City']

## Task
How many station has `victoria` line?

In [65]:
# request data for stations along the Victoria line
victoria_data = request_data(url_victoria)

In [66]:
# count stations along the Victoria Line
victoria_stations = unpack_to_list(victoria_data, 'stopType')
print(f'There are {len(victoria_stations)} stations along the Victoria line.')

There are 16 stations along the Victoria line.


## Task
Plan the journey from Heathrow Airport to Tower Bridge using Bus and Tube? Which way is faster? Example output:
```
Planned duration:
Bus: x minutes
Tube: y minutes
```

We need to search the documentation for correct requests and parameters we need.

In [71]:
# found coordinates using disambiguation results from passing locations: 'Tower Bridge' and 'Heathrow Airport'
journey_to = '51.50599630145,-0.07502752221' # Tower Bridge
journey_from = '51.46962511264,-0.44968473098' # Heathrow Airport

# set parameters for requests.get
# site documention refers to 'bus' mode as 'public-bus'; 'bus' provides better results
journey_bus = {
    ('mode', 'bus'),
    ('journeyPreference', 'leasttime')
}

journey_tube = {
    ('mode', 'tube'),
    ('journeyPreference', 'leasttime')
}

# set URLs using coordinates
url_journey_bus = f'https://api.tfl.gov.uk/Journey/JourneyResults/{journey_from}/to/{journey_to}'
url_journey_tube = f'https://api.tfl.gov.uk/Journey/JourneyResults/{journey_from}/to/{journey_to}'

In [72]:
# request data for quickest routes using tube and bus modes (no authorization)
journey_data_bus = request_data(url_journey_bus, journey_bus, auth=False)
journey_data_tube = request_data(url_journey_tube, journey_tube, auth=False)

In [73]:
# fastest planned durations for each mode
# # bus
bus_routes = []
for route in journey_data_bus['journeys']:
    bus_routes.append(route['duration'])

fastest_bus_route = min(bus_routes)
print(f'Bus: {fastest_bus_route} min')

# # tube
tube_routes = []
for route in journey_data_tube['journeys']:
    tube_routes.append(route['duration'])

fastest_tube_route = min(tube_routes)
print(f'Tube: {fastest_tube_route} min')

Bus: 166 min
Tube: 83 min


In [74]:
# determining which mode is quicker from Heathrow Airport to Tower Bridge
fastest_mode = min(fastest_bus_route, fastest_tube_route)
print(f'Planned duration: {fastest_mode} min')

Planned duration: 83 min


# [end]

In [ ]:
# # set URL vars
# journey_to = '51.50599630145,-0.07502752221' # found using disambiguation results from 'Tower Bridge'
# journey_from = '51.46962511264,-0.44968473098' # found using disambiguation results from 'Heathrow Airport'
# journey_pref = 'leasttime'
# journey_mode_bus = 'bus' # API documention refers to this mode as 'public-bus' but 'bus' works better
# journey_mode_tube = 'tube'

# # urls
# url_journey_bus = f'https://api.tfl.gov.uk/Journey/JourneyResults/{journey_from}/to/{journey_to}?mode={journey_mode_bus}&journeyPreference={journey_pref}'
# url_journey_tube = f'https://api.tfl.gov.uk/Journey/JourneyResults/{journey_from}/to/{journey_to}?mode={journey_mode_tube}&journeyPreference={journey_pref}'